# 概要

本ファイルは、学習済みモデルを使用して、3D物体検出モデルの推論・評価を実行します。 

> 本ファイルで実行するスクリプトはsecond.pytorchをベースとしています。

## Output
本ファイルを実行することで、`results`ディレクトリに、推論結果(result.pkl)を出力します。

```
└ second.pytorc
     └ checkpoints
          └ {model_dir}
                ├ results    <- 本ファイルにて生成されます
                   └ step_xxxx    
                       └─ result.pkl
```

# 準備

あらかじめ、 `data`ディレクトリに、`3d_labels_painted`ディレクトリと、`second.pytorch/checkpoints/{model_dir}` ディレクトリに学習済みモデルがあることを確認してください。

> 本ファイルのデフォルトでは、公開の学習済みモデル`second.pytorch/checkpoints/pointpainting`を設定しています。


```
└ data
    ├ 3d_labels
    └ 3d_labels_painted
        ├ maps
        ├ samples
            ├ CAM_FRONT
            └ LIDAR_TOP
        ├ v1.0-trainval
        ├ gt_database        <- 1-2_create_dataset_metadata.ipynb にて生成
        ├ infos_train.pkl    <- 1-2_create_dataset_metadata.ipynb にて生成
        ├ infos_val.pkl      <- 1-2_create_dataset_metadata.ipynb にて生成
        └ kitti_dbinfos_train.pkl  <- 1-2_create_dataset_metadata.ipynb にて生成
 
├second.pytorch
     └─ checkpoints
          └ {model_dir}
                ├── checkpoints.json
                ├── {model-step}.tckpt <- 学習済みモデル
```

# 推論・評価

検証データ(val)に対して推論と評価を実施します。  

> 学習データと検証データはnuScenesデータセットで定義されています。  
> 検証データはscene-0094とscene-0109 の2シーンで、
> 学習データはそれ以外の35シーンです。

In [1]:
# パスの設定

config_path = "../second.pytorch/second/configs/nuscenes/pointpainting.config"

model_dir = "../second.pytorch/checkpoints/pointpainting"

In [2]:
!python ../second.pytorch/second/pytorch/train.py evaluate \
        --config_path=$config_path \
        --model_dir=$model_dir --measure_time=True --batch_size=4

[  41 1984  992]
Restoring parameters from /work/second.pytorch/checkpoints/pointpainting/voxelnet-24000.tckpt
feature_map_size [1, 248, 124]
Generate output labels...
[100.0%][===================>][2.97it/s][00:15>00:00]   
generate label finished(11.78/s). start eval:
avg example to torch time: 4.511 ms
avg prep time: 41.424 ms
avg voxel_feature_extractor time = 0.699 ms
avg middle forward time = 147.006 ms
avg rpn forward time = 112.753 ms
avg predict time = 29.899 ms
Evaluation nusc                                                                 
Nusc v1.0-trainval Evaluation
car Nusc dist AP@0.5, 1.0, 2.0, 4.0 and TP errors
4.76, 6.25, 7.26, 7.32
trans_err, scale_err, orient_err, vel_err, attr_err: 0.2851, 0.2349, 2.4506, 1.0000, 0.9787
pedestrian Nusc dist AP@0.5, 1.0, 2.0, 4.0 and TP errors
0.00, 0.00, 0.00, 0.00
trans_err, scale_err, orient_err, vel_err, attr_err: 1.0000, 1.0000, 1.0000, 1.0000, 1.0000



## （オプション）推論結果の確認

推論データは pklファイルに保存されています。pklファイルを読み込みことで推論データを確認することができます。

In [3]:
import pickle

result_pkl = "../second.pytorch/checkpoints/pointpainting/eval_results/step_24000/result.pkl"

with open(result_pkl, 'rb') as f:
    result = pickle.load(f)

print("result size: ", len(result))
print(result[0]["box3d_lidar"])  # (n, 7)  [x, y, z, l, h, w, rz]
print(result[0]["scores"])  # (n)
print(result[0]["label_preds"])  # (n)  0: car, 1:pedestrian

result size:  187
tensor([[ 13.5261,  -3.0028,  -0.9795,   1.8182,   4.2835,   1.7755,   1.5868],
        [ 20.1477,   0.6293,  -1.1003,   1.8186,   4.2175,   1.7506,   1.5451],
        [ 11.2831,  -6.6473,  -1.3337,   1.6925,   3.8947,   1.6504,  -1.4244],
        [ 29.8661, -15.1257,  -1.7500,   1.6665,   3.7216,   1.6057,  -1.3606],
        [ 41.2427,   8.1982,  -0.7761,   0.5630,   0.6865,   1.6484,   1.6287],
        [ 20.5918,   8.8067,  -0.5628,   0.5593,   0.6855,   1.6449,  -1.9883]],
       device='cuda:0')
tensor([0.7926, 0.6244, 0.5773, 0.3626, 0.3595, 0.3067], device='cuda:0')
tensor([0, 0, 0, 0, 1, 1], device='cuda:0')


以上